## Todo
- [x] Voxel-based interpolation, reproducing C++ results
- [ ] Replace 0-prediction with knn
- [ ] Experiment with single knn-hybrid approach

In [2]:
import os
import numpy as np
import open3d
import time
from utils.metric import ConfusionMatrix
from utils.point_cloud_util import load_labels, colorize_point_cloud
import multiprocessing

In [7]:
class LabelCounter:
    """
    A simple but non-optimized implementation
    """
    def __init__(self):
        self.labels = []
        self.majority_label = None
        self.finalized = False
        
    def increment(self, label):
        if self.finalized:
            raise RuntimeError("Counter finalized")
        self.labels.append(label)
        
    def get_label(self):
        if not self.finalized:
            self.finalize()
        return self.majority_label
    
    def finalize(self):
        self.majority_label = np.bincount(self.labels).argmax()
        self.finalized = True
    
def get_voxel(point, voxel_size=0.1):
    """
    Returns a voxel tuple
    point: [x, y, z]
    """
    return tuple([np.floor(float(val) / voxel_size) + 0.5 for val in point])

In [8]:
sparse_dir = "/home/ylao/repo/Open3D-PointNet-Semantic/results/sparse"
dense_dir = "/home/ylao/repo/Open3D-PointNet-Semantic/results/dense"
gt_dir = "/home/ylao/data/semantic3d"

In [5]:
file_prefix = "untermaederbrunnen_station1_xyz_intensity_rgb"

# Sparse labels
sparse_labels = np.array(load_labels(os.path.join(sparse_dir, file_prefix + ".labels")), dtype=np.int32)

# Sparse points
sparse_pcd = open3d.read_point_cloud(os.path.join(sparse_dir, file_prefix + ".pcd"))
sparse_points = np.asarray(sparse_pcd.points)

# Dense points
dense_pcd = open3d.read_point_cloud(os.path.join(gt_dir, file_prefix + ".pcd"))
dense_points = np.asarray(dense_pcd.points)

# Ground-trugh dense labels
gt_labels = load_labels(os.path.join(gt_dir, file_prefix + ".labels"))

In [9]:
# Build voxel to label container map
map_voxel_to_label_counter = dict()
for sparse_point, sparse_label in zip(sparse_points, sparse_labels):
    voxel = get_voxel(sparse_point)
    if voxel not in map_voxel_to_label_counter:
        map_voxel_to_label_counter[voxel] = LabelCounter()
    map_voxel_to_label_counter[voxel].increment(sparse_label)
print("{} sparse points, {} registered voxels".format(len(sparse_points), 
                                                      len(map_voxel_to_label_counter)))

3629056 sparse points, 430433 registered voxels


In [14]:
def interpolate_label(dense_index):
    global dense_points
    global sparse_labels
    global map_voxel_to_label_counter
    
    dense_point = dense_points[dense_index]
    voxel = get_voxel(dense_point)
    if voxel not in map_voxel_to_label_counter:
        k, sparse_indexes, _ = sparse_pcd_tree.search_knn_vector_3d(dense_point, 10)
        knn_sparse_labels = sparse_labels[sparse_indexes]
        dense_label = np.bincount(knn_sparse_labels).argmax()
    else:
        dense_label = map_voxel_to_label_counter[voxel].get_label()
    return dense_label

# Build KNN Tree
sparse_pcd_tree = open3d.KDTreeFlann(sparse_pcd)

# Interpolate dense labels
start = time.time()
with multiprocessing.Pool() as pool:
    dense_labels = pool.map(interpolate_label, list(range(len(dense_points))))
print("interpolate_label time: ", time.time() - start)

interpolate_label time:  24.29922604560852


In [15]:
cm = ConfusionMatrix(9)
cm.increment_from_list(gt_labels, dense_labels)
cm.print_metrics()

Confusion matrix:
                  0       1       2       3       4       5       6       7       8 
          0       0   14478      75    1838     996    9111      70   52799     152 
          1       0 13102395    7230     780   34419  504794    6910    2123    4422 
          2       0   21281    3000     777    4975     931    1594     116     282 
          3       0    3011     302  216169   30148   24614    1006     948     119 
          4       0    7654   11044  198519  896597   87592  109258     542   14846 
          5       0  406758     905   24464    3099 8270743   23037   15438   11497 
          6       0  104080    1363   15313  175323 2743399  345658  106019    8582 
          7       0   13810       2     369     297     211     241   71485      67 
          8       0   18086       0       0     810   29209    1554   25181  182512 
IoU per class:
[0.9202572203633537,
 0.05576000892160143,
 0.41849502167309727,
 0.5692234828645129,
 0.68090503002011,
 0.09487401